## Методы решения задачи одномерной минимизации

алгоритмическая сложность, преимущества каждого алгоритма и какой метод предпочтительнее

### 1) Метод Дитохондрии

In [5]:
import pandas as pd
from typing import Callable

def ditohondria(
    f: Callable[[float], float],
    interval: tuple[float,float],
    eps: float   
) -> float:
    table = pd.DataFrame(columns=['a','b','l','x1','x2','f_x1','f_x2'])

    a, b = interval
    while b - a > 2 * eps:
        x1 = (a + b)/2 - eps/2
        x2 = (a + b)/2 + eps/2

        f_x1, f_x2 = f(x1), f(x2)

        table.loc[len(table)] = [a, b, b-a, x1, x2, f_x1, f_x2]

        a, b = (x1, b) if f_x1 > f_x2 else (a, x2)

    table.loc[len(table)] = [a, b, b-a] + [None]*4
    
    return table, (a + b) / 2

### пример ###
f = lambda x: x**2 + 2*x - 4
table, answer = ditohondria(f, (-2, 1), 0.01)

print(table)
print(answer)

          a         b         l        x1        x2      f_x1      f_x2
0 -2.000000  1.000000  3.000000 -0.505000 -0.495000 -4.754975 -4.744975
1 -2.000000 -0.495000  1.505000 -1.252500 -1.242500 -4.936244 -4.941194
2 -1.252500 -0.495000  0.757500 -0.878750 -0.868750 -4.985298 -4.982773
3 -1.252500 -0.868750  0.383750 -1.065625 -1.055625 -4.995693 -4.996906
4 -1.065625 -0.868750  0.196875 -0.972187 -0.962187 -4.999226 -4.998570
5 -1.065625 -0.962187  0.103437 -1.018906 -1.008906 -4.999643 -4.999921
6 -1.018906 -0.962187  0.056719 -0.995547 -0.985547 -4.999980 -4.999791
7 -1.018906 -0.985547  0.033359 -1.007227 -0.997227 -4.999948 -4.999992
8 -1.007227 -0.985547  0.021680 -1.001387 -0.991387 -4.999998 -4.999926
9 -1.007227 -0.991387  0.015840       NaN       NaN       NaN       NaN
-0.9993066406249997


### 2) Метод Фибоначчи

In [2]:
import pandas as pd
from typing import Callable

def fibonacci(
    f: Callable[[float], float],
    interval: tuple[float, float],
    eps: float
) -> float:
    a, b = interval

    table = pd.DataFrame(
        columns=['a','b','l','x1','x2','f_x1','f_x2']
        )
    
    F = [1, 1]
    while F[-1] < (b - a) / eps:
        F.append(F[-1] + F[-2])
    
    delta = (b - a) / F[-1]
    
    N = len(F) - 1
    
    x1 = a + F[N - 2] * delta
    x2 = b - F[N - 2] * delta

    f_x1, f_x2 = f(x1), f(x2)

    table.loc[len(table)] = [a, b, b-a, x1, x2, f_x1, f_x2]

    for k in range(1, N - 2):
        if f_x1 < f_x2:
            b = x2
            x2, f_x2 = x1, f_x1
            x1 = a + F[N - k - 2] * delta
            f_x1 = f(x1)
        else:
            a = x1
            x1, f_x1 = x2, f_x2
            x2 = b - F[N - k - 2] * delta
            f_x2 = f(x2)
        table.loc[len(table)] = [a, b, b-a, x1, x2, f_x1, f_x2]
    
    table.loc[len(table)] = [a, b, b-a] + [None]*4

    return table, (a + b) / 2

f = lambda x: x**2 + 2*x - 4
table, result = fibonacci(f, (-2, 1), 0.1)

print(table)
print(result)

          a         b         l        x1        x2      f_x1      f_x2
0 -2.000000  1.000000  3.000000 -0.852941 -0.147059 -4.978374 -4.272491
1 -2.000000 -0.147059  1.852941 -1.294118 -0.852941 -4.913495 -4.978374
2 -1.294118 -0.147059  1.147059 -0.852941 -0.588235 -4.978374 -4.830450
3 -1.294118 -0.588235  0.705882 -1.029412 -0.852941 -4.999135 -4.978374
4 -1.294118 -0.852941  0.441176 -1.117647 -1.029412 -4.986159 -4.999135
5 -1.117647 -0.852941  0.264706 -1.029412 -0.941176 -4.999135 -4.996540
6 -1.117647 -0.852941  0.264706       NaN       NaN       NaN       NaN
-0.9852941176470587


### 3) Метод золотого сечения

In [3]:
from math import sqrt
from typing import Callable
import pandas as pd

def golden_ratio(
    f: Callable[[float], float],
    interval: tuple[float, float],
    eps: float
) -> float:

    table = pd.DataFrame(
        columns=['a','b','l','x1','x2','f_x1','f_x2']
        )
    
    a, b = interval
    
    alfa = (3 - sqrt(5)) / 2

    x1 = a + alfa * (b - a)
    x2 = b - alfa * (b - a)

    f_x1, f_x2 = f(x1), f(x2)
    
    table.loc[len(table)] = [a, b, b-a, x1, x2, f_x1, f_x2]

    while b - a > 2 * eps:
        if f_x1 > f_x2:
            a = x1
            x1, f_x1 = x2, f_x2
            x2 = b - alfa * (b - a)
            f_x2 = f(x2)
        else:
            b = x2
            x2, f_x2 = x1, f_x1
            x1 = a + alfa * (b - a)
            f_x1 = f(x1)
        table.loc[len(table)] = [a, b, b-a, x1, x2, f_x1, f_x2] 
    
    table.loc[len(table)] = [a, b, b-a] + [None]*4

    return table, (a + b) / 2

f = lambda x: x**2 + 2*x - 4
table, result = golden_ratio(f, (-2, 1), 0.1)

print(table)
print(result)

          a         b         l        x1        x2      f_x1      f_x2
0 -2.000000  1.000000  3.000000 -0.854102 -0.145898 -4.978714 -4.270510
1 -2.000000 -0.145898  1.854102 -1.291796 -0.854102 -4.914855 -4.978714
2 -1.291796 -0.145898  1.145898 -0.854102 -0.583592 -4.978714 -4.826604
3 -1.291796 -0.583592  0.708204 -1.021286 -0.854102 -4.999547 -4.978714
4 -1.291796 -0.854102  0.437694 -1.124612 -1.021286 -4.984472 -4.999547
5 -1.124612 -0.854102  0.270510 -1.021286 -0.957428 -4.999547 -4.998188
6 -1.124612 -0.957428  0.167184 -1.060753 -1.021286 -4.996309 -4.999547
7 -1.124612 -0.957428  0.167184       NaN       NaN       NaN       NaN
-1.0410196624968455


### 4) Метод деления пополам

In [52]:
import pandas as pd
from math import log, ceil
from autograd import grad
from typing import Callable

def bisection_method(
    f: Callable[[float], float],
    interval: tuple[float, float],
    eps: float
):
    table = pd.DataFrame(
        columns=['a','b','l','x','grad_f(x)']
        )
    
    a, b = interval
    
    N = ceil( log(2 * eps / (b - a)) / (log(1) - log(2)) )

    x = 0
    for k in range(0, N):
        x = (a + b) / 2

        grad_f_x = grad(f)(x)

        table.loc[len(table)] = [a, b, b-a, x, grad_f_x] 

        if grad_f_x == 0:
            return x
        elif grad_f_x > 0:
            b = x
        else:
            a = x

    table.loc[len(table)] = [a, b, b-a] + [None] * 2

    return table, (a + b) / 2


f = lambda x: x**2 + 2*x
table, result = bisection_method(f,(-3, 6), 0.1)

print(table.round(4))
print(result)

        a       b       l       x  grad_f(x)
0 -3.0000  6.0000  9.0000  1.5000     5.0000
1 -3.0000  1.5000  4.5000 -0.7500     0.5000
2 -3.0000 -0.7500  2.2500 -1.8750    -1.7500
3 -1.8750 -0.7500  1.1250 -1.3125    -0.6250
4 -1.3125 -0.7500  0.5625 -1.0312    -0.0625
5 -1.0312 -0.7500  0.2812 -0.8906     0.2188
6 -1.0312 -0.8906  0.1406     NaN        NaN
-0.9609375


### 5) Метод Ньютона

In [60]:
from autograd import grad
from typing import Callable
import pandas as pd

def newton_method(
    f: Callable[[float], float],
    x: float,
    eps: float
) -> float:
    table = pd.DataFrame(
        columns=['x0', 'grad_f(x0)','grad2_f(x0)', 'x1']
        )
    
    x1, x0 = x, 0
    
    while abs(x1 - x0) > eps or grad(f)(x1) > eps:
        x0 = x1
        
        grad_f_x0, grad2_f_x0 = grad(f)(x0), grad(grad(f))(x0)
        
        x1 = x0 - grad_f_x0 / grad2_f_x0
        
        table.loc[len(table)] = [x, grad_f_x0, grad2_f_x0, x1]
        
    return table, x1
    
f = lambda x: 4*x**3 - 3*x**4
table, result = newton_method(f, 0.4, 0.001)

print(table)
print(result)

    x0  grad_f(x0)  grad2_f(x0)        x1
0  0.4    1.152000     3.840000  0.100000
1  0.4    0.108000     2.040000  0.047059
2  0.4    0.025324     1.049689  0.022934
3  0.4    0.006167     0.531475  0.011331
4  0.4    0.001523     0.267315  0.005633
5  0.4    0.000379     0.134042  0.002808
6  0.4    0.000094     0.067116  0.001402
7  0.4    0.000024     0.033582  0.000701
0.0007006044051493947


## Методы решения задачи многомерной минимизации

### 6) Метод циклического покооринатного спуска (not)

In [ ]:
from typing import Callable

def cyclic_coordinate_descent(
    f: Callable,
    eps: float
):
    ...

### 7) Метод Хука и Дживса (not)

In [27]:
import numpy as np

norm = np.linalg.norm

### Дано
x = np.array([0.2, 0.1])

d = np.array([0.15, 0.1])

alfa = 2

eps = 0.1

f = lambda x: x[0]**2 + x[0]*x[1] + 2*x[1]**2 + x[0] - x[1]

### Решение
x0 = x.copy()

N = len(x)

d = d * np.eye(N)

for i in range(N):
    if f( x + d[i] ) < f(x):
        x += d[i]
    elif f( x - d[i] ) < f(x):
        x -= d[i]
f(x + (x - x0)) < f(x0)

True

### 8) Метод Роземброка (not)

In [ ]:
eps = 0.1

### 9) Градиентный метод с дроблением шага

In [75]:
import sympy as sym
import numpy as np
import pandas as pd

norm = np.linalg.norm

def gradient_splitting_step(
    f: sym.Add,
    x0: np.ndarray,
    alfa: float,
    d: float
):
    x1 = x0

    alfa_start_value = alfa

    table = pd.DataFrame(
        columns=['x0','f_x','grad_f_x (s)','alfa','x1','f_x','-d*alfa*norm(s)','f(x1)-f(x) > -d*alfa*norm(s)'])

    # переменные
    variables = tuple(f.free_symbols)

    # сортировка переменных по числу
    variables = sorted(variables, key=
                       lambda var: int(sym.srepr(var)[-3]))

    # градиент функции по всем переменным
    gradients_f = [
        sym.diff(f, variable) for variable in variables]

    # превращение из символьного представления в функцию
    gradients_f = [
        sym.lambdify(variables, gradient_f) 
                   for gradient_f in gradients_f]
    
    # преващение из символьного представления в функцию
    f = sym.lambdify(variables, f)
    
    for k in range(1000):
        alfa = alfa_start_value
        
        # градиент в точке (или s)
        grad_f_x = - np.array([f(*x1) for f in gradients_f])

        x = x1 + alfa * grad_f_x

        while f(*x) - f(*x1) >= - d * alfa * norm(grad_f_x)**2:
            table.loc[len(table)] = [
                x0, f(*x0), grad_f_x, alfa, x, f(*x), -d*alfa*norm(grad_f_x)**2, True
            ]
            
            alfa /= 2
            
            x = x1 + alfa * grad_f_x

        table.loc[len(table)] = [
                x0, f(*x0), grad_f_x, alfa, x, f(*x), -d*alfa*norm(grad_f_x)**2, False
            ]

        x1 = x
        
        if norm(x - x0) <= eps:
            table.loc[len(table)] = [x1, f(*x1)] + [None]*6
            return table, x1
        
        x0 = x1

x = np.array([0, 0])
alfa = 1
d = 0.1
eps = 0.1

x1, x2 = sym.symbols('x1 x2')
f = x1**2 + x1*x2 + 2*x2**2 + x1 - x2

table, result = gradient_splitting_step(f, x, alfa, d)

print(table.round(3))
print(result)

                  x0    f_x     grad_f_x (s)  alfa                 x1    f_x  \
0             [0, 0]  0.000          [-1, 1]  1.00            [-1, 1]  0.000   
1             [0, 0]  0.000          [-1, 1]  0.50        [-0.5, 0.5] -0.500   
2        [-0.5, 0.5] -0.500     [-0.5, -0.5]  1.00        [-1.0, 0.0]  0.000   
3        [-0.5, 0.5] -0.500     [-0.5, -0.5]  0.50      [-0.75, 0.25] -0.500   
4        [-0.5, 0.5] -0.500     [-0.5, -0.5]  0.25    [-0.625, 0.375] -0.562   
5    [-0.625, 0.375] -0.562  [-0.125, 0.125]  1.00       [-0.75, 0.5] -0.562   
6    [-0.625, 0.375] -0.562  [-0.125, 0.125]  0.50  [-0.6875, 0.4375] -0.570   
7  [-0.6875, 0.4375] -0.570             None   NaN               None    NaN   

   -d*alfa*norm(s) f(x1)-f(x) > -d*alfa*norm(s)  
0           -0.200                         True  
1           -0.100                        False  
2           -0.050                         True  
3           -0.025                         True  
4           -0.013           

C:\Users\Azerty\AppData\Local\Temp\ipykernel_2884\522973379.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  table.loc[len(table)] = [x1, f(*x1)] + [None]*6


### 10) Метод наискорейшего спуска

In [171]:
import sympy as sym
import numpy as np

norm = np.linalg.norm

def get_alfa_from_x_and_gradient(f):
    # переменные
    variables = tuple(f.free_symbols)

    # сортировка переменных по числу: x1,x2,...,xn
    variables = sorted(variables, key=
                       lambda var: int(sym.srepr(var)[-3]))

    N = len(variables)

    # градиент функции по всем переменным
    gradients_f = [
        sym.diff(f, variable) for variable in variables]

    a = sym.symbols('a')

    # составляем переменные s_i для каждого x_i
    s_variables = list(sym.symbols(' '.join([
        f's{sym.srepr(variable)[-3]}' for variable in variables
    ])))

    # [(x_i, a*s_i + x_i)] - лист кортежей с заменой
    replacement = [(variables[i], variables[i] + a * s_variables[i]) 
                    for i in range(N)]

    # градиент функции по всем переменным
    gradients_f = [
        sym.diff(f, variable) for variable in variables]

    # заменяем переменные в градиенте
    gradients_f = [
        f.subs(replacement) for f in gradients_f
    ]

    # домножаем градиент по каждой переменной на s_i
    expr = sum([gradients_f[i] * s_variables[i] 
                for i in range(N)])
    
    # выражаем a
    solution = sym.solve(expr, a)[0]

    return sym.lambdify(variables + s_variables, solution)

def get_gradient(f, x0):
    # переменные
    variables = tuple(f.free_symbols)

    # сортировка переменных по числу
    variables = sorted(variables, key=
                       lambda var: int(sym.srepr(var)[-3]))

    # градиент функции по всем переменным
    gradients_f = [
        sym.diff(f, variable) for variable in variables]

    # превращение из символьного представления в функцию
    lambd_gradients_f = [
        sym.lambdify(variables, gradient_f) 
                    for gradient_f in gradients_f]
    
    grad_f_x = np.array([f(*x0) for f in lambd_gradients_f])

    return np.where(np.abs(grad_f_x) > 0.0001, grad_f_x, 0)

def steepest_descent(
    f: sym.Add,
    x0: np.ndarray,
    eps: float
):
    x1 = x0

    alfa = get_alfa_from_x_and_gradient(f)

    for k in range(1000):
        grad_f_x = get_gradient(f, x1)
        
        alfa_ = alfa(*x1, *-grad_f_x)

        x1 = x0 + alfa_ * -grad_f_x

        if norm(x1 - x0) < eps:
            return x1
        x0 = x1

### Дано
x1, x2 = sym.symbols('x1 x2')
f = x1**2 + x1*x2 + 2*x2**2 + x1 - x2

x0 = np.array([0, 0])
eps = 0.1

steepest_descent(f, x0, eps)

array([-0.6875,  0.4375])

### 11) Метод Ньютона

In [172]:
import sympy as sym
from itertools import product
import numpy as np

def Newton(
    f: sym.Add,
    x0: np.ndarray,
    eps: float
):
    x1 = x0

    variables = tuple(f.free_symbols)

    # сортировка переменных по числу
    variables = sorted(variables, key=
                        lambda var: int(sym.srepr(var)[-3]))

    N = len(variables)

    pairs = np.array(
        [var for var in product(variables,repeat=2)])

    gesse = np.array(
        [f.diff(*pair) for pair in pairs], dtype='float64'
    ).reshape((N,N))

    gesse_inverse = np.linalg.inv(gesse)

    alfa = get_alfa_from_x_and_gradient(f)

    for k in range(1000):
        grad_f_x = get_gradient(f, x1)

        s = -(gesse_inverse @ grad_f_x)

        alfa_ = alfa(*x1, *s)

        alfa_ = alfa_ if abs(alfa_) > 0.001 else 0

        x1 = x0 + alfa_ * s

        if norm(x1 - x0) < eps:
            return x1
        
        x0 = x1

### Дано
x1, x2 = sym.symbols('x1 x2')
f = x1**2 + x1*x2 + 2*x2**2 + x1 - x2

x0 = np.array([0, 0])
eps = 0.1

Newton(f,x0, eps)


<lambdifygenerated-10534>:2: RuntimeWarning: invalid value encountered in scalar divide
  return (1/2)*(-2*s1*x1 - s1*x2 - s1 - s2*x1 - 4*s2*x2 + s2)/(s1**2 + s1*s2 + 2*s2**2)


array([-0.71428571,  0.42857143])

### Оставшиеся
1. Метод циклического покооринатного спуска
2. Метод Хука и Дживса
3. Метод Роземброка

## Условная оптимизация

### 12) 

### Оставшиеся
1. методы возможных направлений
2. метод зойтендейка
3. метод проекции градиента розена
4. метод приведенного градиента вулфа
5. методы штрафных и барьерных функций